In [ ]:
#install some requirements

!pip install split-folders
!pip uninstall opencv-python-headless
!pip uninstall albumentations
!pip install opencv-python-headless==4.1.2.30
!pip install -U git+https://github.com/albumentations-team/albumentations

Found existing installation: opencv-python-headless 4.1.2.30
Uninstalling opencv-python-headless-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless-4.1.2.30.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/cv2/.libs/libQtCore-bbdab771.so.4.8.7
    /usr/local/lib/python3.7/dist-packages/cv2/.libs/libQtGui-903938cd.so.4.8.7
    /usr/local/lib/python3.7/dist-packages/cv2/.libs/libQtTest-1183da5d.so.4.8.7
Proceed (y/n)? y
  Successfully uninstalled opencv-python-headless-4.1.2.30
Found existing installation: albumentations 1.1.0
Uninstalling albumentations-1.1.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/albumentations-1.1.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/albumentations/*
Proceed (y/n)? y
  Successfully uninstalled albumentations-1.1.0
  Using cached opencv_python_headless-4.1.2.30-cp37-cp37m-manylinux1_x86_64

  Cloning https://github.com/albumentations-team/albumentations to /tmp/pip-req-build-81m64tey
  Running command git clone -q https://github.com/albumentations-team/albumentations /tmp/pip-req-build-81m64tey
  Created wheel for albumentations: filename=albumentations-1.1.0-py3-none-any.whl size=112723 sha256=6be3f0692c422b193c04107322487ccc806d321b8baa2aa6d0a55c23da945828
  Stored in directory: /tmp/pip-ephem-wheel-cache-4bfs_mdd/wheels/3a/25/ed/ec3b518e7a332d7f0a3bb37c280e1b784cf2f79b94b3c7d00b
Successfully built albumentations


In [ ]:
import ee
import os
import cv2
import glob
import os.path
from pathlib import Path
import re
import splitfolders
import albumentations as A

In [ ]:
 #authentication of google earth engine to access some files as example the local relief model
 ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=PLoWjId3yzN7pFqgENKl-hrtm6444NmoSyF_dTh63Oo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjNFsA7vsr7iCkDwqJqNbFmb6GY2c5aZg17eI5rN8TiumQ2IEqnJPk

Successfully saved authorization token.


In [ ]:
#initialization of google earth engine
ee.Initialize()

In [ ]:
LRM = ee.Image('users/fabriziobotelho/tiff_files')
geometry = LRM.geometry()
lrm_info = LRM.getInfo()
print(lrm_info) #debug

#geo-coordinates of upper left and lower right corners of LRM image
upper_left = [-8.907277129492169,42.16335245423417]
lower_right = [-8.06051504207768,41.59280385023923]

points = ee.Geometry.MultiPoint([upper_left, lower_right])


{'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [69977, 63086], 'crs': 'EPSG:3763', 'crs_transform': [1, 0, -63978.5, 0, -1, 277085.5]}, {'id': 'b2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [69977, 63086], 'crs': 'EPSG:3763', 'crs_transform': [1, 0, -63978.5, 0, -1, 277085.5]}, {'id': 'b3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [69977, 63086], 'crs': 'EPSG:3763', 'crs_transform': [1, 0, -63978.5, 0, -1, 277085.5]}, {'id': 'b4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [69977, 63086], 'crs': 'EPSG:3763', 'crs_transform': [1, 0, -63978.5, 0, -1, 277085.5]}], 'version': 1644944174622534, 'id': 'users/fabriziobotelho/tiff_files', 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-8.900443739793436, 41.59280407757193], [-8.90044

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#this function gets the dimention in pixel of LRM image
def getDimentionsImage(img):
  imgDescription = ee.Algorithms.Describe(img)
  w = ee.List(ee.Dictionary(ee.List(ee.Dictionary(imgDescription).get("bands")).get(0)).get("dimensions")).get(0)
  w = w.getInfo()
  h = ee.List(ee.Dictionary(ee.List(ee.Dictionary(imgDescription).get("bands")).get(0)).get("dimensions")).get(1)
  h = h.getInfo()

  return w, h

In [ ]:
width, height = getDimentionsImage(LRM)

print('width:', width, 'height:', height)

width: 69977 height: 63086


In [ ]:
x_geo1 = upper_left[0]
y_geo1 = upper_left[1]
x_geo2 = lower_right[0]
y_geo2 = lower_right[1]

x1 = 0;
y1 = 0;
x2 = width - 1 
y2 = height - 1

#equations to convert geo-coordinates to pixel coordinates
#x = x_geo * m1 + b1
#y = y_geo * m2 + b2

m1 = (x1 - x2)/(x_geo1 - x_geo2)
b1 = x1 - x_geo1 * m1
print('m1:', m1, 'b1:', b1)

m2 = (y1 - y2)/(y_geo1 - y_geo2)
b2 = y1 - y_geo1 * m2
print('m2:', m2, 'b2:', b2)

m1: 82639.5052873298 b1: 736092.9754383798
m2: -110569.01998932785 b2: 4661960.560329294


In [ ]:
#this function uses the equations before obtained to convert a geo-coordinate to pixel coordinate
def convert_GeoCoord_to_PixelCoord(geocoordinate, m1, m2, b1, b2):
  x_geo = geocoordinate[0]
  y_geo = geocoordinate[1]
  x = round(x_geo * m1 + b1)
  y = round(y_geo * m2 + b2)
  
  return x, y

In [ ]:
#check the minimum and maximum coordinate of LRM image
print('upper_left_pixels:', convert_GeoCoord_to_PixelCoord(upper_left, m1, m2, b1, b2))
print('lower_right_pixels:', convert_GeoCoord_to_PixelCoord(lower_right, m1, m2, b1, b2))

upper_left_pixels: (0, 0)
lower_right_pixels: (69976, 63085)


In [ ]:
#read the shape files witch contains the geo-coordinates of archaeological objects
shape_arcos = ee.FeatureCollection('users/fabriziobotelho/MamoasArcos')
shape_arcos = shape_arcos.geometry()
coordinates_arcos = shape_arcos.coordinates().getInfo()

shape_lab = ee.FeatureCollection('users/fabriziobotelho/MamoasLaboreiro')
shape_lab = shape_lab.geometry();
coordinates_lab = shape_lab.coordinates().getInfo()

print('Mamoas arcos coordinates:', coordinates_arcos)
print('length:', len(coordinates_arcos))
print('Mamoas laboreiro coordinates:', coordinates_lab)
print('length:', len(coordinates_lab))

Mamoas arcos coordinates: [[-8.443945496150414, 41.96002943539938], [-8.444422619963133, 41.95940962035295], [-8.445746972976004, 41.959075187773934], [-8.447164967111004, 41.95924463361397], [-8.392549897408395, 41.8905655591888], [-8.275476199250173, 41.97359402080396], [-8.270892243367232, 41.97545346594325], [-8.270575647192437, 41.97586816234122], [-8.269228998674299, 41.976938346594046], [-8.262134568898265, 41.99370010745395], [-8.261929450249806, 41.99408359014454], [-8.317177712302, 41.877424588384294], [-8.485071785166344, 41.84396795318029], [-8.418876430027941, 41.81337851995365], [-8.300126109873624, 41.87799089421808], [-8.300607692787395, 41.8513611427269], [-8.31359259505503, 41.87844572252554], [-8.317592408699971, 41.88951767044124], [-8.24084236136806, 42.008954692157786], [-8.24111436653232, 42.00840622272821], [-8.240793311256473, 42.010274586069606], [-8.240815606761739, 42.01068036426547], [-8.333658549795539, 41.948498200075164], [-8.500607293236548, 41.94226437

In [ ]:
#this function is the inverse of the previous function, with it we can get the geo-coordinates that corresponds to the pixel coordinates
def convert_PixelCoord_to_GeoCoord(pixelcoordinate, m1, m2, b1, b2):
  x = pixelcoordinate[0]
  y = pixelcoordinate[1]
  x_geo = (x - b1)/m1
  y_geo = (y - b2)/m2
  
  return x_geo, y_geo

In [ ]:
#check if the previous functions works correctly
print(convert_PixelCoord_to_GeoCoord((0,0), m1, m2, b1, b2))
print(convert_PixelCoord_to_GeoCoord((69976,63086), m1, m2, b1, b2))

(-8.907277129492169, 42.16335245423417)
(-8.06051504207768, 41.592794806114576)


In [ ]:
#example to verify if works correctly
upper_left = [-8.907277129492169,42.16335245423417]
lower_right = [-8.06051504207768,41.59280385023923]

values = convert_GeoCoord_to_PixelCoord(upper_left, m1, m2, b1, b2)
print(values)
values = convert_GeoCoord_to_PixelCoord(lower_right, m1, m2, b1, b2)
print(values)


(0, 0)
(69976, 63085)


In [ ]:
#initialy the resolution was 640 by 640 pixels, but the objective is get the images with less archaeological objects. So the resolution used is 320 pixels
#and allow get more images that is good
resolution = 320

number_y_points = ee.Number(height).divide(resolution).round().add(1)
numberColumns = round((width + 1) / resolution)
numberRows = round((height + 1)/ resolution)
print('numberColumns, n_y_points:', numberColumns, numberRows)

numberColumns, n_y_points: 219 197


In [ ]:
arcos_pixel = []
lab_pixel = []

#for loops to convert the coordinates of archaeological objets to pixels coordinates
for i in coordinates_arcos:
  arcos_pixel.append(convert_GeoCoord_to_PixelCoord(i, m1, m2, b1, b2))

print('geo coordinates:', coordinates_arcos)
print('pixel_coordinates:', arcos_pixel) 
print('length:', len(arcos_pixel))

for i in coordinates_lab:
  lab_pixel.append(convert_GeoCoord_to_PixelCoord(i, m1, m2, b1, b2))

print('geo coordinates:', coordinates_lab)
print('pixel_coordinates:', lab_pixel) 
print('length:', len(lab_pixel))  

geo coordinates: [[-8.443945496150414, 41.96002943539938], [-8.444422619963133, 41.95940962035295], [-8.445746972976004, 41.959075187773934], [-8.447164967111004, 41.95924463361397], [-8.392549897408395, 41.8905655591888], [-8.275476199250173, 41.97359402080396], [-8.270892243367232, 41.97545346594325], [-8.270575647192437, 41.97586816234122], [-8.269228998674299, 41.976938346594046], [-8.262134568898265, 41.99370010745395], [-8.261929450249806, 41.99408359014454], [-8.317177712302, 41.877424588384294], [-8.485071785166344, 41.84396795318029], [-8.418876430027941, 41.81337851995365], [-8.300126109873624, 41.87799089421808], [-8.300607692787395, 41.8513611427269], [-8.31359259505503, 41.87844572252554], [-8.317592408699971, 41.88951767044124], [-8.24084236136806, 42.008954692157786], [-8.24111436653232, 42.00840622272821], [-8.240793311256473, 42.010274586069606], [-8.240815606761739, 42.01068036426547], [-8.333658549795539, 41.948498200075164], [-8.500607293236548, 41.94226437680245], 

In [ ]:
#this function counts the number of archaeological objects inside the block image, and we just want the images with this objects
def CountNumberOfMamoas(Xmin, Ymin, Xmax, Ymax):
  count = 0
  arr1 = []
  arr2 = []
  center = []

  #read mamoas of arcos (51 mamoas)
  for i in arcos_pixel:
    x = i[0]
    y = i[1]

    if x >= Xmin and x <= Xmax and y >= Ymin and y <= Ymax:
      count = count + 1
      arr1.append((x - Xmin))
      arr2.append((y - Ymin))
      center = list(zip(arr1, arr2))

  #read mamoas of laboreiro (85 mamoas)
  for i in lab_pixel:
    x = i[0]
    y = i[1]
    if x >= Xmin and x <= Xmax and y >= Ymin and y <= Ymax:
      count = count + 1
      arr1.append((x - Xmin))
      arr2.append((y - Ymin))
      center = list(zip(arr1, arr2))

  return count, center

In [ ]:
#just to check and debug
n, c = CountNumberOfMamoas(0, 0, 70000, 70000)
print(n)

136


In [ ]:
#In this part of code is constructed a block that will go through the LRM image and crop/export the image inside that block 
#if the number of archaelogical objects is greater than zero
#so the images cropped has the 320x320 of dimension and the scale used was 1 meter by pixel, that means each image corresponds to 320 x 320 meters on terrain

#initialize parameters
x11 = 0
x12 = 0
x21 = resolution
x22 = 0
x31 = resolution
x32 = resolution
x41 = 0
x42 = resolution

i = 0
j = 0
k = 0
allPoints = []

while i < numberRows:
  while j < numberColumns:

    x1 = convert_PixelCoord_to_GeoCoord([x11, x12], m1, m2, b1, b2)
    x2 = convert_PixelCoord_to_GeoCoord([x21, x22], m1, m2, b1, b2)
    x3 = convert_PixelCoord_to_GeoCoord([x31, x32], m1, m2, b1, b2)
    x4 = convert_PixelCoord_to_GeoCoord([x41, x42], m1, m2, b1, b2)

    nMamoas, centerPoint = CountNumberOfMamoas(x11, x12, x31, x32)

    block = ee.Geometry.Polygon([[x1[0], x1[1]], [x2[0], x2[1]], [x3[0], x3[1]], [x4[0], x4[1]]])


    if nMamoas > 0:
      #print('Image: ' + str(k) + ' has ' + str(nMamoas) + ' mamoas')
      task = ee.batch.Export.image.toDrive(**{
        'image': DTM,
        'description': str(k) + "_" + str(nMamoas),
        'folder': 'dataset_mamoas_res_320',
        'scale': 1, #1m/px
        'region': block
        })
      img = cv2.imread("/content/drive/MyDrive/dataset_mamoas_res_320/" + str(k) + "_" + str(nMamoas) + ".tif")

      k = k + 1
      task.start()

      for point in centerPoint:
        img = cv2.circle(img, point, radius = 2, color = (0, 255, 0), thickness = -1)
        xmin = point[0] - 10
        xmax = point[0] + 10
        ymin = point[1] - 10
        ymax = point[1] + 10
        img = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 1)
        cv2.imwrite("/content/drive/MyDrive/dataset_mamos_res_320_check/" + str(k) + ".tif", img)
        
      allPoints.append(centerPoint)

    x11 = x21
    x12 = x22
    x21 = x21 + resolution
    x22 = x22
    x41 = x31
    x42 = x42 
    x31 = x31 + resolution
    x32 = x32

    j = j + 1
  
  x11 = 0
  x21 = 0
  x41 = 0
  x31 = 0

  x12 = x12 + resolution
  x22 = x22 + resolution
  x42 = x42 + resolution
  x32 = x32 + resolution
  j = 0
  i = i + 1


In [ ]:
#obtain bounding boxes with georeferenced data
#we have the center point of each archaological object (mamoa) converted to pixel coordinates 
#we have the limits xmin, xmax, ymin, ymax of each original image cropped of LRM

#resize images to 640x640 and convert to jpg format
src_path = "/content/drive/MyDrive/dataset_mamoas_res_320/*.tif" 
dst_path = "/content/drive/MyDrive/dataset_mamoas_res_320_resized_to_640/data_30_30/"

#dimentions of to resize the images
width = 640
height = 640
dim = (width, height)

##obtain new coordinates of images after resized
w_bbox = 30 #15, 20, 22, 25, 30 
h_bbox = 30 #15, 20, 22, 25, 30 
width_yolo = w_bbox / width  #to normalize
heigth_yolo = h_bbox / height #to normalize

ratio = 640 / 320 #note that now we have images twice as big

classID = 0 #just have a mamoa

name_of_image = []
k = 0

for i in glob.glob(src_path):
  img = cv2.imread(i, cv2.IMREAD_UNCHANGED)

  resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
  
  name_of_image.append(Path(i).stem)

  cv2.imwrite(str(dst_path) + str(name_of_image[k]) + ".jpg", resized)
  #print('Resized Dimensions : ', resized.shape)
  k = k + 1

k = 0

for points in allPoints:

  info = [] #initialize list

  # Write files
  filename = os.path.join(dst_path, str(name_of_image[k]) +".txt")  
  file = open(filename, "w")

  for j in points:
    x_yolo = round(j[0] * ratio) / width
    y_yolo = round(j[1] * ratio) / height

    #print(classID, x_yolo, y_yolo, width_yolo, heigth_yolo)
    
    info.append([classID, x_yolo, y_yolo, width_yolo, heigth_yolo])

  for item in info:
    text = re.sub(r"[\([{,})\]]", "", str(item))
    file.write("%s\n" % text)

  k = k + 1

  file.close()


In [ ]:
#split the files into 60, 20, 20, whitch corresponds to train, validation and test

srcpath = "/content/drive/MyDrive/dataset_mamoas_res_320_resized_to_640/"
outpath = "/content/drive/MyDrive/dataset_mamoas/"
splitfolders.ratio(srcpath, output=outpath,
    seed=1337, ratio=(.6, .2, .2), group_prefix=2, move=False) 

#remove the txt files from test file
dir_name = "/content/drive/MyDrive/dataset_mamoas/test/data/"
test = os.listdir(dir_name)

for item in test:
    if item.endswith(".txt"):
        os.remove(os.path.join(dir_name, item))

In [ ]:
##########################################################################################
#reference: https://github.com/albumentations-team/albumentations/issues/459#issuecomment-734454278
#change: /usr/local/lib/python3.6/dist-packages/albumentations/augmentations/bbox_utils.py
#in function checkBlock():
#my added block 
#bbox=list(bbox)
#for i in range(4):
  #if (bbox[i]<0) :
    #bbox[i]=0
  #elif (bbox[i]>1) :
    #bbox[i]=1
#bbox=tuple(bbox)
#end of block
##########################################################################################

#dataset augmentation
src_path = "/content/drive/MyDrive/dataset_mamoas_res_320_resized_to_640/data/"
dst_path = "/content/drive/MyDrive/augmentation/augm_bbox_30_30/"

ratio = round(640 / 320)
classID = 0 #mamoa

info = []

k = 0
for points in allPoints:

  bboxes = []
  for j in points:
    x = (j[0] * ratio) / width
    y = (j[1] * ratio) / height
    bboxes.append([x , y, width_yolo, heigth_yolo])

  image = cv2.imread(src_path + str(name_of_image[k]) + ".jpg")

  transform = A.Compose(
  [
      A.Transpose(p=0.5),
      A.HorizontalFlip(p=0.5),
      A.VerticalFlip(p=0.5),
      A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25, p = 0.9),
      A.OneOf([
            A.Blur(blur_limit = 3, p = 0.5),
            A.ColorJitter(p = 0.5),    
      ], p = 1.0), 
  ], bbox_params = A.BboxParams(format = "yolo", label_fields = [])
  )

  for i in range(6):
    augmentations = transform(image = image, bboxes = bboxes)
    augmented_img = augmentations["image"]
    cv2.imwrite(dst_path + str(name_of_image[k]) + "_" + str(i) + ".jpg", augmented_img)

    # Write files
    filename = os.path.join(dst_path, str(name_of_image[k]) + "_" + str(i) + ".txt")  
    file = open(filename, "w")

    for item in augmentations["bboxes"]:
      text = re.sub(r"[\([{,})\]]", "", str(item))
      file.write(str(classID) + " " + "%s\n" % text)
    file.close()
  k = k + 1


In [ ]:
#split the augmented images/files

srcpath = "/content/drive/MyDrive/augmentation/"
outpath = "/content/drive/MyDrive/dataset_mamoas/"
splitfolders.ratio(srcpath, output=outpath,
    seed=1337, ratio=(.8, .2), group_prefix=2, move=False) 

Copying files: 2400 files [01:07, 35.66 files/s]
